# ProxyWhirl - Quick Start Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wyattowalsh/proxywhirl/blob/main/examples.ipynb)

**ProxyWhirl** is an advanced Python proxy rotation library with auto-fetching, validation, and persistence.

This notebook demonstrates:
- Installing proxywhirl
- Creating proxy pools
- Rotating through proxies with different strategies

## 1. Installation

In [ ]:
# Install proxywhirl from PyPI
!pip install -q proxywhirl

# Verify installation
import proxywhirl

print(f"ProxyWhirl v{proxywhirl.__version__} installed!")

## 2. Create a Proxy Pool

Define proxies manually. In production, you'd use real proxy addresses.

In [ ]:
from proxywhirl import Proxy, ProxyPool

# Create sample proxies (use your own proxy addresses in production)
proxies = [
    Proxy(url="http://proxy1.example.com:8080", country_code="US"),
    Proxy(url="http://proxy2.example.com:8080", country_code="DE"),
    Proxy(url="http://proxy3.example.com:8080", country_code="JP"),
    Proxy(url="http://proxy4.example.com:8080", country_code="GB"),
    Proxy(url="http://proxy5.example.com:8080", country_code="FR"),
]

# Create a pool
pool = ProxyPool(name="demo-pool", proxies=proxies)
print(f"Pool created with {pool.size} proxies")

# Show pool contents
for p in pool.get_all_proxies():
    print(f"  - {p.url} ({p.country_code})")

## 3. Rotation Strategies

ProxyWhirl supports multiple rotation strategies:
- **RoundRobinStrategy** - Cycle through proxies in order
- **RandomStrategy** - Random selection
- **WeightedStrategy** - Based on proxy weights
- **LeastUsedStrategy** - Prefer less-used proxies
- **PerformanceBasedStrategy** - Based on response times
- **GeoTargetedStrategy** - Select by geographic location
- **SessionPersistenceStrategy** - Maintain sessions
- **CostAwareStrategy** - Optimize for cost

In [ ]:
from proxywhirl import RandomStrategy, RoundRobinStrategy, SelectionContext

# Create a round-robin strategy
strategy = RoundRobinStrategy()

print("Round-robin rotation:")
for i in range(7):  # More than pool size to show cycling
    proxy = strategy.select(pool, SelectionContext())
    print(f"  Request {i + 1}: {proxy.url}")

In [ ]:
# Try random strategy
random_strategy = RandomStrategy()

print("Random rotation:")
for i in range(5):
    proxy = random_strategy.select(pool, SelectionContext())
    print(f"  Request {i + 1}: {proxy.url}")

## 4. ProxyWhirl Class

The `ProxyWhirl` class combines a pool with a strategy and provides HTTP request methods.

In [ ]:
from proxywhirl import ProxyWhirl

# Create rotator with our proxies
rotator = ProxyWhirl(proxies=[p.model_copy(deep=True) for p in pool.get_all_proxies()])

print(f"Rotator ready with {rotator.pool.size} proxies")
print(f"Strategy: {rotator.strategy.__class__.__name__}")

## 5. Pool Statistics

In [ ]:
# Get pool statistics
stats = rotator.get_pool_stats()
print("Pool Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

## 6. Change Strategy at Runtime

In [ ]:
# Switch to random strategy
rotator.set_strategy("random")
print(f"Strategy changed to: {rotator.strategy.__class__.__name__}")

# Switch to least-used
rotator.set_strategy("least-used")
print(f"Strategy changed to: {rotator.strategy.__class__.__name__}")

# Back to round-robin
rotator.set_strategy("round-robin")
print(f"Strategy changed to: {rotator.strategy.__class__.__name__}")

## 7. Making HTTP Requests

With real proxies, you can make HTTP requests like this:

```python
# The rotator automatically rotates through proxies
response = rotator.get("https://httpbin.org/ip")
print(response.json())

# POST request
response = rotator.post("https://httpbin.org/post", json={"key": "value"})
```

The rotator includes:
- Automatic retry with exponential backoff
- Circuit breaker pattern for failing proxies
- Connection pooling for performance

## Next Steps

- **Documentation:** [proxywhirl.readthedocs.io](https://proxywhirl.readthedocs.io)
- **GitHub:** [github.com/wyattowalsh/proxywhirl](https://github.com/wyattowalsh/proxywhirl)
- **PyPI:** [pypi.org/project/proxywhirl](https://pypi.org/project/proxywhirl)

### Features to Explore:
- Async client (`AsyncProxyWhirl`)
- SQLite persistence for proxy storage
- Circuit breaker for failing proxies
- Retry with exponential backoff
- Multi-tier caching
- REST API server
- CLI tools (`proxywhirl --help`)